In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display   #for loading and visualizing audio files
import IPython.display as ipd   #to play audio

plt.style.use('ggplot')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import time 
import pickle

# Notes

Start writing up presentation as you go after modelling stage

## 3 methods:

**A.** PCA each feature (spectrogram) ---> save each PCA'd feature into own dataframe ---> model on each one individually --> majority vote predictions

**B.** From PCA'd dataframe combine into one big and model entirely ---> predictions

**C.** Do descriptive stats and model on this ---> predictions


- simple model on various features count predictions and choose from majority vote

(chroma, tempo, spectogram, mse)
- MAKE SURE each AUDIO file is same sample rate!


- turn into black & white images, pca

**N. of frames from hop length and y**

If you divide your y_shape length by the hop_length - you get the number of the frames. 

- y.shape/hop_length

**Regarding warning message :**

There is nothing to fix: this is operating as intended.

Again, this is not an error, it is a warning. Your code will work just fine, it just will fall back on the (slower) audioread decoder. This is unavoidable for now, since libsndfile still does not support mp3.

If you're really concerned about speed, you could transcode your mp3s into a more permissive format (eg ogg vorbis

In [ ]:
'''
chroma_stft([y, sr, S, norm, n_fft, …])            Compute a chromagram from a waveform or power spectrogram.

chroma_cqt([y, sr, C, hop_length, fmin, …])        Constant-Q chromagram

chroma_cens([y, sr, C, hop_length, fmin, …])       Computes the chroma variant “Chroma Energy Normalized” (CENS)

melspectrogram([y, sr, S, n_fft, …])               Compute a mel-scaled spectrogram.

mfcc([y, sr, S, n_mfcc, dct_type, norm, lifter])   Mel-frequency cepstral coefficients (MFCCs)

rms([y, S, frame_length, hop_length, …])           Compute root-mean-square (RMS) value for each frame, either from the audio samples y or from a spectrogram S.

spectral_centroid([y, sr, S, n_fft, …])            Compute the spectral centroid.

spectral_bandwidth([y, sr, S, n_fft, …])           Compute p’th-order spectral bandwidth.

spectral_contrast([y, sr, S, n_fft, …])            Compute spectral contrast

spectral_flatness([y, S, n_fft, hop_length, …])    Compute spectral flatness

spectral_rolloff([y, sr, S, n_fft, …])             Compute roll-off frequency.

poly_features([y, sr, S, n_fft, hop_length, …])    Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram.

tonnetz([y, sr, chroma])                           Computes the tonal centroid features (tonnetz)

zero_crossing_rate(y[, frame_length, …])           Compute the zero-crossing rate of an audio time series.
'''